# Filtro EMA Adattivo (Scalare e Circolare) per SCL3300
## Analisi Teorica, Modellazione Stocastica, Implementazione C e Integrazione LoRa

> **Scopo** — Documento accademico completo (in italiano) che unisce teoria, modellazione su $S^1$ per variabili angolari, implementazione in C pronta per firmware MCU con sensore **SCL3300**, logging **UART** e telemetria **LoRa**.

## Indice

1. [Introduzione e Contesto](#1-introduzione-e-contesto)  
2. [Modello di Misura e Calibrazione degli Offset](#2-modello-di-misura-e-calibrazione-degli-offset)  
3. [EMA Scalare a α Costante: Dominio z, Risposta in Frequenza, Varianza](#3-ema-scalare-a-α-costante-dominio-z-risposta-in-frequenza-varianza)  
4. [EMA Adattivo (Scalare): Innovazione, Stime Esponenziali, Mappatura u↦α](#4-ema-adattivo-scalare-innovazione-stime-esponenziali-mappatura-uα)  
5. [EMA Adattivo Circolare su S¹: Wrapping, Media Circolare, Aggiornamento](#5-ema-adattivo-circolare-su-s1-wrapping-media-circolare-aggiornamento)  
6. [Stabilità, Ritardo, Considerazioni Numeriche e Δt Variabile](#6-stabilità-ritardo-considerazioni-numeriche-e-δt-variabile)  
7. [Implementazione in C: Strutture, Init, Update](#7-implementazione-in-c-strutture-init-update)  
8. [Pipeline Firmware con SCL3300 e Payload LoRa](#8-pipeline-firmware-con-scl3300-e-payload-lora)  
9. [Tuning dei Parametri: Linee Guida e Trade-off](#9-tuning-dei-parametri-linee-guida-e-trade-off)  
10. [Debug e Troubleshooting](#10-debug-e-troubleshooting)  
11. [Estensioni e Varianti](#11-estensioni-e-varianti)  
12. [Bibliografia Essenziale](#12-bibliografia-essenziale)  
13. [Licenza](#13-licenza)

## 1. Introduzione e Contesto

Un filtro **EMA** (Exponential Moving Average) è un filtro IIR del primo ordine che produce una media esponenziale del segnale. In ambito embedded è preferito per **semplicità**, **stabilità** e **costo computazionale** contenuto. Per applicazioni dinamiche conviene renderlo **adattivo**: lento in quiete (massimo smoothing) e rapido quando l'ingresso varia (minima latenza).

In questo documento si trattano due casi:
- **Scalare** (es. temperatura): EMA adattivo con $α_k$ determinato dall'innovazione normalizzata.
- **Circolare** (angoli in gradi $θ_{X,Y,Z}$): EMA adattivo su **$S^1$** con wrapping $(-180,180]$ e media con $\cos/\sin$.

**Pipeline logica:**

```
┌─────────────┐     ┌─────────────────────┐     ┌──────────────────────────┐     ┌───────────────┐
│   SCL3300   │ --> │ Calibrazione Offset │ --> │ Filtro EMA Adattivo     │ --> │ UART / LoRa   │
│   (SPI)     │     │ (angoli X,Y,Z)      │     │  - Scalare (T)          │     │  (output)     │
│             │     │                     │     │  - Circolare (X,Y,Z)    │     │               │
└─────────────┘     └─────────────────────┘     └──────────────────────────┘     └───────────────┘
```

## 2. Modello di Misura e Calibrazione degli Offset

### 2.1 Modello di misura

Per un canale generico (es. asse X):
$$x_k = x_k^{\text{vero}} + b + n_k,$$
dove $b$ è un **bias** statico e $n_k$ rumore a media nulla, varianza $\sigma^2$.

### 2.2 Stima del bias (media campionaria)

Durante la calibrazione a fermo (posa nota), raccogli $N$ campioni $\{x_i\}$. La stima dell'offset è
$$o_X = \frac{1}{N}\sum_{i=1}^{N} x_i,\qquad
o_Y = \frac{1}{N}\sum_{i=1}^{N} y_i,\qquad
o_Z = \frac{1}{N}\sum_{i=1}^{N} z_i.$$

Le misure compensate:
$$\tilde{x}_k = x_k - o_X,\qquad
\tilde{y}_k = y_k - o_Y,\qquad
\tilde{z}_k = z_k - o_Z.$$

### 2.3 Errore statistico della stima del bias

Se $n_k$ è i.i.d. con varianza $\sigma^2$:
$$\mathrm{Var}[\bar{x}] = \frac{\sigma^2}{N},\qquad
\mathrm{std}(\bar{x}) = \frac{\sigma}{\sqrt{N}}.$$
Tipico: $N\in[100,300]$.

## 3. EMA Scalare a α Costante: Dominio z, Risposta in Frequenza, Varianza

### 3.1 Ricorrenza e funzione di trasferimento

Ricorrenza:
$$y_k = (1-\alpha)y_{k-1} + \alpha x_k,\qquad 0<\alpha\le 1.$$

Trasformata-$z$:
$$H(z)=\frac{Y(z)}{X(z)}=\frac{\alpha}{1-(1-\alpha)z^{-1}}.$$

Polo $p=1-\alpha$ (stabilità BIBO: $|p|<1 \Rightarrow 0<\alpha<2$; con $\alpha\in(0,1]$ è sempre stabile).

### 3.2 Risposta impulsiva e interpretazione

$$h[k]=\alpha(1-\alpha)^k\,u[k],\qquad \sum_{k=0}^{\infty} h[k]=1.$$
L'EMA è una **media** con pesi esponenziali.

### 3.3 Risposta in frequenza (modulo e fase)

$$H(e^{j\omega})=\frac{\alpha}{1-(1-\alpha)e^{-j\omega}}.$$

$$|H(e^{j\omega})|=\frac{\alpha}{\sqrt{1-2(1-\alpha)\cos\omega+(1-\alpha)^2}}.$$

Fase:
$$\phi(\omega)=\tan^{-1}\!\left(\frac{(1-\alpha)\sin\omega}{\,1-(1-\alpha)\cos\omega\,}\right).$$

### 3.4 Varianza in uscita per rumore bianco

Se $x_k\sim \mathcal{WN}(0,\sigma^2)$, in regime stazionario
$$\mathrm{Var}[y_k]=\frac{\alpha}{2-\alpha}\,\sigma^2
\approx \frac{\alpha}{2}\,\sigma^2\quad(\alpha\ll 1).$$

### 3.5 Ritardo medio (group delay) e finestra equivalente

$$\tau_d \approx \frac{1-\alpha}{\alpha}\,\frac{\Delta t}{2},
\qquad
N_{\text{eff}} \approx \frac{2}{\alpha}-1.$$

## 4. EMA Adattivo (Scalare): Innovazione, Stime Esponenziali, Mappatura u↦α

### 4.1 Innovazione e stime esponenziali

$$\begin{aligned}
e_k &= x_k - y_{k-1},\\
m_k &= \beta m_{k-1} + (1-\beta)\,e_k,\\
d_k &= e_k - m_k,\\
s_k^2 &= \beta s_{k-1}^2 + (1-\beta)\,d_k^2,
\end{aligned}$$
con $0<\beta<1$ e finestra equivalente $N_{\text{eff},\beta}\approx \frac{1}{1-\beta}$.

### 4.2 Innovazione normalizzata e mappatura non lineare

$$u_k=\frac{|d_k|}{\sqrt{s_k^2}+\varepsilon},\qquad
\alpha_k=\alpha_{\min}+(\alpha_{\max}-\alpha_{\min})\,\frac{u_k^p}{1+u_k^p},$$
con $0<\alpha_{\min}\le\alpha_{\max}\le 1$ e $p\ge 1$ (tipico $p=2$).

### 4.3 Aggiornamento adattivo

$$y_k=(1-\alpha_k)\,y_{k-1}+\alpha_k\,x_k
= y_{k-1}+\alpha_k\,(x_k-y_{k-1}).$$

**Intuizione:** in quiete $u_k\!\approx\!0\Rightarrow\alpha_k\!\approx\!\alpha_{\min}$ (smoothing forte); in transitorio $u_k\!\gg\!1\Rightarrow\alpha_k\!\to\!\alpha_{\max}$ (reattività).

## 5. EMA Adattivo Circolare su S¹: Wrapping, Media Circolare, Aggiornamento

### 5.1 Wrapping e distanza angolare minima

Intervallo canonico in gradi: $(-180,180]$. Per un errore angolare "grezzo" $\delta=\theta-\hat{\theta}$:
$$e^\theta = \mathrm{wrap}(\delta) =
\begin{cases}
\delta+360 & \text{se }\delta\le -180,\\
\delta-360 & \text{se }\delta> 180,\\
\delta & \text{altrimenti.}
\end{cases}$$

### 5.2 Adattività sull'errore angolare wrapped

$$\begin{aligned}
m_k^\theta &= \beta m_{k-1}^\theta + (1-\beta)\,e_k^\theta,\\
d_k^\theta &= e_k^\theta - m_k^\theta,\\
(s_k^\theta)^2 &= \beta (s_{k-1}^\theta)^2 + (1-\beta)\,(d_k^\theta)^2,\\
u_k^\theta &= \frac{|d_k^\theta|}{\sqrt{(s_k^\theta)^2}+\varepsilon},\\
\alpha_k^\theta &= \alpha_{\min}+(\alpha_{\max}-\alpha_{\min})\frac{(u_k^\theta)^p}{1+(u_k^\theta)^p}.
\end{aligned}$$

### 5.3 Media circolare tramite cos/sin

$$\begin{aligned}
c_k &= (1-\alpha_k^\theta)\,c_{k-1} + \alpha_k^\theta \cos(\theta_k),\\
s_k &= (1-\alpha_k^\theta)\,s_{k-1} + \alpha_k^\theta \sin(\theta_k),\\
y_k^\theta &= \operatorname{atan2}(s_k,\,c_k).
\end{aligned}$$

Per robustezza numerica normalizzare $(c_k,s_k)$ se $r_k=\sqrt{c_k^2+s_k^2}$ è molto piccolo.  
Questa formulazione evita i "salti" a $\pm 180^\circ$ (es. $179^\circ\to-179^\circ$ trattato come $+2^\circ$).

## 6. Stabilità, Ritardo, Considerazioni Numeriche e Δt Variabile

- **Stabilità:** $\alpha_k\in[\alpha_{\min},\alpha_{\max}]\subset(0,1] \Rightarrow$ polo $1-\alpha_k\in[0,1)$ ⇒ BIBO stabile.  
- **Ritardo:** in quiete (piccola $\alpha_k$) il ritardo aumenta; in transitorio (grande $\alpha_k$) il ritardo si riduce.  
- **$\Delta t$ variabile:** si può usare $\alpha(\Delta t)=1-e^{-\Delta t/\tau}$ e $\beta(\Delta t)=e^{-\Delta t/\tau_\beta}$.  
- **Numerica:** usare `float` con funzioni `*f` (`sinf`, `cosf`, `atan2f`, `sqrtf`, `powf`). $\varepsilon\sim10^{-6}$ evita div/0.  

## 7. Implementazione in C: Strutture, Init, Update

### 7.1 Strutture dati

```c
typedef struct {
    float m;           // media dell'innovazione
    float s2;          // varianza dell'innovazione
    float alpha_min, alpha_max;
    float beta;
    float p;
    float eps;
    float y;           // uscita filtrata
    float alpha;       // alpha corrente
} EMA1_Adaptive;

typedef struct {
    EMA1_Adaptive core;
    float c, s;        // media circolare cos/sin
    float y;           // angolo filtrato [gradi]
} EMA1_CircularDeg;
```

### 7.2 Funzioni principali

- `ema1_adaptive_init()`: Inizializzazione filtro scalare
- `ema1_adaptive_update()`: Aggiornamento filtro scalare
- `ema1_circdeg_init()`: Inizializzazione filtro circolare
- `ema1_circdeg_update()`: Aggiornamento filtro circolare
- `wrap_deg()`: Wrapping angoli in $(-180,180]$

**Parametri di partenza:**  
- Angoli $(X,Y,Z)$: $\alpha_{\min}=0.04$, $\alpha_{\max}=0.55$, $\beta=0.990$, $p=2$, $\varepsilon=10^{-6}$  
- Temperatura $T$: $\alpha_{\min}=0.02$, $\alpha_{\max}=0.20$, $\beta=0.995$, $p=2$, $\varepsilon=10^{-6}$

## 8. Pipeline Firmware con SCL3300 e Payload LoRa

### 8.1 Sequenza operativa

1. **Init SCL3300** (SPI, WHOAMI/STATUS)  
2. **Calibrazione offset**: acquisisci $N$ campioni a fermo $\to$ calcola $o_X,o_Y,o_Z$  
3. **Init filtri** con valori compensati  
4. **Loop**: leggi registri $\to$ compensa $\to$ aggiorna filtri $\to$ log/LoRa

### 8.2 Esempio init/loop

```c
#define A_MIN  0.04f
#define A_MAX  0.55f
#define BETA   0.990f
#define P_PWR  2.0f
#define EPS_   1e-6f

EMA1_CircularDeg fx, fy, fz;
EMA1_Adaptive    ftemp;

void filters_init(float X0, float Y0, float Z0, float T0) {
    ema1_circdeg_init(&fx, X0, A_MIN, A_MAX, BETA, P_PWR, EPS_);
    ema1_circdeg_init(&fy, Y0, A_MIN, A_MAX, BETA, P_PWR, EPS_);
    ema1_circdeg_init(&fz, Z0, A_MIN, A_MAX, BETA, P_PWR, EPS_);
    ema1_adaptive_init(&ftemp, T0, 0.02f, 0.20f, 0.995f, 2.0f, 1e-6f);
}
```

### 8.3 Log UART (esempio)

```
RAW  X:  12.34  Y:  -0.80  Z: 179.10 |
FILT X: 12.20  Y: -0.77  Z: -179.5   |
αx:0.056 αy:0.041 αz:0.310 | T: 27.8°C (αT:0.045)
```

### 8.4 Payload LoRa (8 byte)

Quantizzazione in centigradi e centi-°C:
$$q_\theta = \mathrm{round}(100\,\theta_f),\qquad q_T=\mathrm{round}(100\,T_f).$$

```c
int16_t Xc = (int16_t)lrintf(Xf * 100.0f);
int16_t Yc = (int16_t)lrintf(Yf * 100.0f);
int16_t Zc = (int16_t)lrintf(Zf * 100.0f);
int16_t Tc = (int16_t)lrintf(Tf * 100.0f);

uint8_t buf[8];
memcpy(&buf[0], &Xc, 2);
memcpy(&buf[2], &Yc, 2);
memcpy(&buf[4], &Zc, 2);
memcpy(&buf[6], &Tc, 2);
```

## 9. Tuning dei Parametri: Linee Guida e Trade-off

| Parametro | Ruolo | Tipico | Effetto |
|---|---|---|---|
| $\alpha_{\min}$ | Smoothing in quiete | 0.02–0.08 | Più piccolo $\Rightarrow$ più liscio (più ritardo) |
| $\alpha_{\max}$ | Reattività in transitorio | 0.3–0.7 | Più alto $\Rightarrow$ più veloce (meno smoothing) |
| $\beta$ | Memoria stime $m,s^2$ | 0.98–0.995 | Alto $\Rightarrow$ stima più stabile ma lenta |
| $p$ | Ripidità $u\to\alpha$ | 2 | Più alto $\Rightarrow$ transizione più brusca |
| $\varepsilon$ | Stabilizzatore | $10^{-6}$ | Evita div/0 |

Procedura rapida: calibra offset → verifica $\alpha$ in quiete → prova step rapidi → regola $\alpha_{\max}$, $\beta$, $p$.

## 10. Debug e Troubleshooting

- **$\alpha$ non scende in quiete:** aumenta $\beta$ o riduci $\alpha_{\min}$  
- **Filtro lento su transitori:** aumenta $\alpha_{\max}$ o riduci $\beta$  
- **Salti a $\pm 180^\circ$:** assicurati di usare wrap nell'innovazione e media circolare  
- **Overflow/LoRa:** controlla scaling e clamp prima del cast  
- **Norma $(c,s)$ troppo piccola:** normalizza per evitare perdita di direzione  

## 11. Estensioni e Varianti

- $\alpha(\Delta t)$ e $\beta(\Delta t)$ per campionamenti irregolari  
- Clipping "soft" su $u$ per outlier estremi  
- Telemetria dell'indice di affidabilità: invia $\alpha$ quantizzato su 0–255  
- Cascata di due EMA (2° ordine) con adattività condivisa  
- Statistiche circolari aggiuntive: lunghezza del risultante $R=\sqrt{c^2+s^2}$  

## 12. Bibliografia Essenziale

- S. Haykin, *Adaptive Filter Theory*, Prentice Hall  
- A. V. Oppenheim, R. W. Schafer, *Discrete-Time Signal Processing*, Prentice Hall  
- A. Papoulis, *Probability, Random Variables, and Stochastic Processes*, McGraw-Hill  
- K. V. Mardia, P. E. Jupp, *Directional Statistics*, Wiley  
- U. Jammalamadaka, A. Sengupta, *Topics in Circular Statistics*, World Scientific  

## 13. Licenza

© 2025 — Rilasciato con licenza **MIT** (adattabile secondo necessità del progetto).